# Uploading the dataset

##First upload the dataset manually using the "Files" section in the sidebar. The names of the test and train files should be "text.txt" and "train.txt". This block reads the files and create csv files according to uploaded files.

#### *Note: If you want to run this block again first remove train and test csv files.

#### *Note: You should do these steps for each runtime as google colab recycles allocated resources every 12 hours####

  

# Intalling required dependancies, Importing required libraries and nltk modules

In [1]:
!pip install gensim smart_open scikit-learn nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 req

In [ ]:
import glob
import json
import os
import numpy as np
import nltk
import sklearn
import collections
import smart_open
import random
import gensim
import time
import difflib
import string

from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from gensim.models import KeyedVectors
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import LinearRegression


nltk.download('wordnet_ic')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


ModuleNotFoundError: No module named 'gensim'

#Removing index in file and adding it to `new_test.txt` and `new_train.txt`

In [ ]:
for filename in glob.glob('*.txt'):
  with open(filename, 'r') as f:
    lines = list(f.readlines())
    name = filename.split(".txt")[0]
    if "test" == name or "train" == name:
      with open("/content/new_" + name + ".txt", "a+") as file:
        for line in lines[1:]:
          splitted = line.split("\t")
          file.write(splitted[1] + "\t" + splitted[2] + "\t" + splitted[3])

# Downloading Google News corpus (3 billion running words) word vector model (3 million 300-dimension English word vectors).

In [ ]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!gzip -d /content/GoogleNews-vectors-negative300.bin.gz

--2025-06-26 16:49:48--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.32.158, 52.217.96.14, 52.216.222.80, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.32.158|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-06-26 16:49:48 ERROR 404: Not Found.

gzip: /content/GoogleNews-vectors-negative300.bin.gz: No such file or directory


#Initializing word vector

In [ ]:
vec1 = KeyedVectors.load_word2vec_format("/content/GoogleNews-vectors-negative300.bin", binary=True)

#Extract sentence similarity based on vectors

In [ ]:
def extract_res_vec_similarity(s1, s2):
    first_sents_embeddings = np.empty([0,300])
    second_sents_embeddings = np.empty([0,300])

    first_vecs = np.array([])
    for w in s1.split():
        w = w.strip('?.,')
        if w in vec1:
            first_vec = np.array([vec1[w]])
            if first_vecs.shape[0] == 0:
                first_vecs = first_vec
            else:
                first_vecs = np.vstack((first_vecs, first_vec))
        else:
            if first_vecs.shape[0] == 0:
                first_vecs = np.random.normal(0, 5, 300)
            else:
                first_vecs = np.vstack((first_vecs, np.random.normal(0, 5, 300)))
        # print("first ")
        # print(first_vecs.shape)
    if(first_vecs.shape == (300, )):
        temp = first_vecs
    else:
        temp = np.mean(first_vecs, axis=0)
    # print(temp.shape)
    first_sents_embeddings = np.append(first_sents_embeddings, [temp], axis=0)

    second_vecs = np.array([])
    for w in s2.split():
        w = w.strip('?.,')
        if w in vec1:
            second_vec = np.array([vec1[w]])
            if second_vecs.shape[0] == 0:
                second_vecs = second_vec
            else:
                second_vecs = np.vstack((second_vecs, second_vec))
        else:
            if second_vecs.shape[0] == 0:
                second_vecs = np.random.normal(0, 5, 300)
            else:
                second_vecs = np.vstack((second_vecs, np.random.normal(0, 5, 300)))
        # print("second ")
        # print(second_vecs.shape)
    if(second_vecs.shape == (300,)):
        temp = second_vecs
    else:
        temp = np.mean(second_vecs, axis=0)
    # print(temp.shape)
    second_sents_embeddings = np.append(second_sents_embeddings, [temp], axis=0)

    for i in range(len(first_sents_embeddings)):
        # cosine similarity

        ret = np.dot(first_sents_embeddings[i], second_sents_embeddings[i]) / (np.linalg.norm(first_sents_embeddings[i]) * np.linalg.norm(second_sents_embeddings[i]))
        ret = 5*(ret + 1) / 2

    return ret

#InferSent
##*Note: This section uses InferSent to calculate sentence similiraty score based on it's pre-trained models which are not available for public access at the moment you can upload infersent1 or infersnet2 locally, if you don't have access to these files skip to the next section

In [ ]:
!curl -Lo infersent2.pkl https://dl.fbaipublicfiles.com/senteval/infersent/infersent2.pkl
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip glove.840B.300d.zip

In [ ]:
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

"""
This file contains the definition of encoders used in https://arxiv.org/pdf/1705.02364.pdf
"""

import numpy as np
import time

import torch
import torch.nn as nn


class InferSent(nn.Module):

    def __init__(self, config):
        super(InferSent, self).__init__()
        self.bsize = config['bsize']
        self.word_emb_dim = config['word_emb_dim']
        self.enc_lstm_dim = config['enc_lstm_dim']
        self.pool_type = config['pool_type']
        self.dpout_model = config['dpout_model']
        self.version = 1 if 'version' not in config else config['version']

        self.enc_lstm = nn.LSTM(self.word_emb_dim, self.enc_lstm_dim, 1,
                                bidirectional=True, dropout=self.dpout_model)

        assert self.version in [1, 2]
        if self.version == 1:
            self.bos = '<s>'
            self.eos = '</s>'
            self.max_pad = True
            self.moses_tok = False
        elif self.version == 2:
            self.bos = '<p>'
            self.eos = '</p>'
            self.max_pad = False
            self.moses_tok = True

    def is_cuda(self):
        # either all weights are on cpu or they are on gpu
        return self.enc_lstm.bias_hh_l0.data.is_cuda

    def forward(self, sent_tuple):
        # sent_len: [max_len, ..., min_len] (bsize)
        # sent: (seqlen x bsize x worddim)
        sent, sent_len = sent_tuple

        # Sort by length (keep idx)
        sent_len_sorted, idx_sort = np.sort(sent_len)[::-1], np.argsort(-sent_len)
        sent_len_sorted = sent_len_sorted.copy()
        idx_unsort = np.argsort(idx_sort)

        idx_sort = torch.from_numpy(idx_sort).cuda() if self.is_cuda() \
            else torch.from_numpy(idx_sort)
        sent = sent.index_select(1, idx_sort)

        # Handling padding in Recurrent Networks
        sent_packed = nn.utils.rnn.pack_padded_sequence(sent, sent_len_sorted)
        sent_output = self.enc_lstm(sent_packed)[0]  # seqlen x batch x 2*nhid
        sent_output = nn.utils.rnn.pad_packed_sequence(sent_output)[0]

        # Un-sort by length
        idx_unsort = torch.from_numpy(idx_unsort).cuda() if self.is_cuda() \
            else torch.from_numpy(idx_unsort)
        sent_output = sent_output.index_select(1, idx_unsort)

        # Pooling
        if self.pool_type == "mean":
            sent_len = torch.FloatTensor(sent_len.copy()).unsqueeze(1).cuda()
            emb = torch.sum(sent_output, 0).squeeze(0)
            emb = emb / sent_len.expand_as(emb)
        elif self.pool_type == "max":
            if not self.max_pad:
                sent_output[sent_output == 0] = -1e9
            emb = torch.max(sent_output, 0)[0]
            if emb.ndimension() == 3:
                emb = emb.squeeze(0)
                assert emb.ndimension() == 2

        return emb

    def set_w2v_path(self, w2v_path):
        self.w2v_path = w2v_path

    def get_word_dict(self, sentences, tokenize=True):
        # create vocab of words
        word_dict = {}
        sentences = [s.split() if not tokenize else self.tokenize(s) for s in sentences]
        for sent in sentences:
            for word in sent:
                if word not in word_dict:
                    word_dict[word] = ''
        word_dict[self.bos] = ''
        word_dict[self.eos] = ''
        return word_dict

    def get_w2v(self, word_dict):
        assert hasattr(self, 'w2v_path'), 'w2v path not set'
        # create word_vec with w2v vectors
        word_vec = {}
        with open(self.w2v_path, encoding='utf-8') as f:
            for line in f:
                word, vec = line.split(' ', 1)
                if word in word_dict:
                    word_vec[word] = np.fromstring(vec, sep=' ')
        print('Found %s(/%s) words with w2v vectors' % (len(word_vec), len(word_dict)))
        return word_vec

    def get_w2v_k(self, K):
        assert hasattr(self, 'w2v_path'), 'w2v path not set'
        # create word_vec with k first w2v vectors
        k = 0
        word_vec = {}
        with open(self.w2v_path, encoding='utf-8') as f:
            for line in f:
                word, vec = line.split(' ', 1)
                if k <= K:
                    word_vec[word] = np.fromstring(vec, sep=' ')
                    k += 1
                if k > K:
                    if word in [self.bos, self.eos]:
                        word_vec[word] = np.fromstring(vec, sep=' ')

                if k > K and all([w in word_vec for w in [self.bos, self.eos]]):
                    break
        return word_vec

    def build_vocab(self, sentences, tokenize=True):
        assert hasattr(self, 'w2v_path'), 'w2v path not set'
        word_dict = self.get_word_dict(sentences, tokenize)
        self.word_vec = self.get_w2v(word_dict)
        print('Vocab size : %s' % (len(self.word_vec)))

    # build w2v vocab with k most frequent words
    def build_vocab_k_words(self, K):
        assert hasattr(self, 'w2v_path'), 'w2v path not set'
        self.word_vec = self.get_w2v_k(K)
        print('Vocab size : %s' % (K))

    def update_vocab(self, sentences, tokenize=True):
        assert hasattr(self, 'w2v_path'), 'warning : w2v path not set'
        assert hasattr(self, 'word_vec'), 'build_vocab before updating it'
        word_dict = self.get_word_dict(sentences, tokenize)

        # keep only new words
        for word in self.word_vec:
            if word in word_dict:
                del word_dict[word]

        # udpate vocabulary
        if word_dict:
            new_word_vec = self.get_w2v(word_dict)
            self.word_vec.update(new_word_vec)
        else:
            new_word_vec = []
        print('New vocab size : %s (added %s words)'% (len(self.word_vec), len(new_word_vec)))

    def get_batch(self, batch):
        # sent in batch in decreasing order of lengths
        # batch: (bsize, max_len, word_dim)
        embed = np.zeros((len(batch[0]), len(batch), self.word_emb_dim))

        for i in range(len(batch)):
            for j in range(len(batch[i])):
                embed[j, i, :] = self.word_vec[batch[i][j]]

        return torch.FloatTensor(embed)

    def tokenize(self, s):
        from nltk.tokenize import word_tokenize
        if self.moses_tok:
            s = ' '.join(word_tokenize(s))
            s = s.replace(" n't ", "n 't ")  # HACK to get ~MOSES tokenization
            return s.split()
        else:
            return word_tokenize(s)

    def prepare_samples(self, sentences, bsize, tokenize, verbose):
        sentences = [[self.bos] + s.split() + [self.eos] if not tokenize else
                     [self.bos] + self.tokenize(s) + [self.eos] for s in sentences]
        n_w = np.sum([len(x) for x in sentences])

        # filters words without w2v vectors
        for i in range(len(sentences)):
            s_f = [word for word in sentences[i] if word in self.word_vec]
            if not s_f:
                import warnings
                warnings.warn('No words in "%s" (idx=%s) have w2v vectors. \
                               Replacing by "</s>"..' % (sentences[i], i))
                s_f = [self.eos]
            sentences[i] = s_f

        lengths = np.array([len(s) for s in sentences])
        n_wk = np.sum(lengths)
        if verbose:
            print('Nb words kept : %s/%s (%.1f%s)' % (
                        n_wk, n_w, 100.0 * n_wk / n_w, '%'))

        # sort by decreasing length
        lengths, idx_sort = np.sort(lengths)[::-1], np.argsort(-lengths)
        sentences = np.array(sentences)[idx_sort]

        return sentences, lengths, idx_sort

    def encode(self, sentences, bsize=64, tokenize=True, verbose=False):
        tic = time.time()
        sentences, lengths, idx_sort = self.prepare_samples(
                        sentences, bsize, tokenize, verbose)

        embeddings = []
        for stidx in range(0, len(sentences), bsize):
            batch = self.get_batch(sentences[stidx:stidx + bsize])
            if self.is_cuda():
                batch = batch.cuda()
            with torch.no_grad():
                batch = self.forward((batch, lengths[stidx:stidx + bsize])).data.cpu().numpy()
            embeddings.append(batch)
        embeddings = np.vstack(embeddings)

        # unsort
        idx_unsort = np.argsort(idx_sort)
        embeddings = embeddings[idx_unsort]

        if verbose:
            print('Speed : %.1f sentences/s (%s mode, bsize=%s)' % (
                    len(embeddings)/(time.time()-tic),
                    'gpu' if self.is_cuda() else 'cpu', bsize))
        return embeddings

    def visualize(self, sent, tokenize=True):

        sent = sent.split() if not tokenize else self.tokenize(sent)
        sent = [[self.bos] + [word for word in sent if word in self.word_vec] + [self.eos]]

        if ' '.join(sent[0]) == '%s %s' % (self.bos, self.eos):
            import warnings
            warnings.warn('No words in "%s" have w2v vectors. Replacing \
                           by "%s %s"..' % (sent, self.bos, self.eos))
        batch = self.get_batch(sent)

        if self.is_cuda():
            batch = batch.cuda()
        output = self.enc_lstm(batch)[0]
        output, idxs = torch.max(output, 0)
        # output, idxs = output.squeeze(), idxs.squeeze()
        idxs = idxs.data.cpu().numpy()
        argmaxs = [np.sum((idxs == k)) for k in range(len(sent[0]))]

        # visualize model
        import matplotlib.pyplot as plt
        x = range(len(sent[0]))
        y = [100.0 * n / np.sum(argmaxs) for n in argmaxs]
        plt.xticks(x, sent[0], rotation=45)
        plt.bar(x, y)
        plt.ylabel('%')
        plt.title('Visualisation of words importance')
        plt.show()

        return output, idxs

In [ ]:
from nltk.corpus import wordnet_ic
from gensim.models import KeyedVectors
import torch
import nltk
nltk.download('punkt')

V = 2
MODEL_PATH = '/content/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

brown_ic = wordnet_ic.ic('ic-brown.dat')
nltk.download('punkt')
infersent.set_glove_path('/content/glove.840B.300d.txt')
infersent.build_vocab_k_words(K=100000)

In [ ]:

def extract_sent2vec_similarity(s1, s2):
    eb1 = infersent.encode([s1], tokenize=True)
    eb2 = infersent.encode([s2], tokenize=True)
    a = np.squeeze(eb1).tolist()
    b = np.squeeze(eb2).tolist()
    ret = 1 - spatial.distance.cosine(a,b)
    return ret

#Simple baseline
##For the simple baseline an unsupervised approach by creating sentence vectors with each dimension representing whether an individual word appears in a sentence is used. The final score is calculated using cosine similairty between the sentence vectors.

In [ ]:
def sentence_similarity_simple_baseline(s1, s2,counts = None):
    def embedding_count(s):
        ret_embedding = defaultdict(int)
        for w in s.split():
            w = w.strip('?.,')
            ret_embedding[w] += 1
        return ret_embedding
    first_sent_embedding = embedding_count(s1)
    second_sent_embedding = embedding_count(s2)
    Embedding1 = []
    Embedding2 = []
    if counts:
        for w in first_sent_embedding:
            Embedding1.append(first_sent_embedding[w] * 1.0/ (counts[w]+0.001))
            Embedding2.append(second_sent_embedding[w] *1.0/ (counts[w]+0.001))
    else:
        for w in first_sent_embedding:
            Embedding1.append(first_sent_embedding[w])
            Embedding2.append(second_sent_embedding[w])
    ret_score = 0
    if not 0 == sum(Embedding2):
        sm= difflib.SequenceMatcher(None,Embedding1,Embedding2)
        ret_score = sm.ratio()*5
    return ret_score

#Word alignment
##Look for each word of sentence A in Wordnet, we can get a synonym set (synset). Then align this word with a corresponding one in sentence B by using path similarity in Wordnet trees. The labeled alignments are used as one useful feature.

In [ ]:
# =========== util func ==============
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'): return 'n'
    if tag.startswith('V'): return 'v'
    if tag.startswith('J'): return 'a'
    if tag.startswith('R'): return 'r'
    return None

def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None

# =========== feature extraction ==============
def sentence_similarity_word_alignment(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet and ppdb """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
    score, count = 0.0, 0
    ppdb_score, align_cnt = 0, 0
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        L = [synset.path_similarity(ss) for ss in synsets2]
        L_prime = L
        L = [l for l in L if l]
        if L:
            best_score = max(L)
            score += best_score
            count += 1
    # Average the values
    if count >0: score /= count

    # ppdb_wa_pen_ua features
    # len_s1, len_s2 = len(sentence1), len(sentence2)
    # ppdb_score = (1 - 0.4 * (len_s1 + len_s2 - 2*align_cnt) / float(len_s1 + len_s2)) * ppdb_score
    return score#, ppdb_score



#Unigram Overlap
##Count the unigram overlap between sentence A and sentence B, with synonym check. Then Calculate the ration of overlap count over the total length of two sentences.

In [ ]:
def extract_overlap_pen(s1, s2):
    """
    :param s1:
    :param s2:
    :return: overlap_pen score
    """
    ss1 = s1.strip().split()
    ss2 = s2.strip().split()
    ovlp_cnt = 0
    for w1 in ss1:
        ovlp_cnt += ss2.count(w1)
    score = 2 * ovlp_cnt / (len(ss1) + len(ss2) + .0)
    return score



#Absolute Difference
##Let Cta and Cta be the counts of tokens of type t ∈ {all tokens, adjectives, adverbs, nouns, and verbs} in sentence A and B respectively. Then calculate the absolute difference as |Cta−Ctb|/(Cta+Ctb).

In [ ]:
def extract_absolute_difference(s1, s2):
    """t \in {all tokens, adjectives, adverbs, nouns, and verbs}"""
    s1, s2 = word_tokenize(s1), word_tokenize(s2)
    pos1, pos2 = pos_tag(s1), pos_tag(s2)
    # all tokens
    t1 = abs(len(s1) - len(s2)) / float(len(s1) + len(s2))
    # all adjectives
    cnt1 = len([1 for item in pos1 if item[1].startswith('J')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('J')])
    if cnt1 == 0 and cnt2 == 0:
        t2 = 0
    else:
        t2 = abs(cnt1 - cnt2) / float(cnt1 + cnt2)
    # all adverbs
    cnt1 = len([1 for item in pos1 if item[1].startswith('R')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('R')])
    if cnt1 == 0 and cnt2 == 0:
        t3 = 0
    else:
        t3 = abs(cnt1 - cnt2) / float(cnt1 + cnt2)
    # all nouns
    cnt1 = len([1 for item in pos1 if item[1].startswith('N')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('N')])
    if cnt1 == 0 and cnt2 == 0:
        t4 = 0
    else:
        t4 = abs(cnt1 - cnt2) / float(cnt1 + cnt2)
    # all verbs
    cnt1 = len([1 for item in pos1 if item[1].startswith('V')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('V')])
    if cnt1 == 0 and cnt2 == 0:
        t5 = 0
    else:
        t5 = abs(cnt1 - cnt2) / float(cnt1 + cnt2)
    return [t1, t2, t3, t4, t5]



#Min to Max Ratio
##Let Ct1 and Ct2 be the counts of type t ∈ {all, adjectives, adverbs, nouns, and verbs} for shorter Sentence 1 and longer Sentence 2 respectively. then calculate the minimum to maximum ratio Ct1/Ct2 as one feature.

In [ ]:
def extract_mmr_t(s1, s2):
    shorter = 1
    if(len(s1) > len(s2)):  shorter = 2

    s1, s2 = word_tokenize(s1), word_tokenize(s2)
    pos1, pos2 = pos_tag(s1), pos_tag(s2)
    # all tokens
    t1 = (len(s1)+0.001) / (len(s2) +0.001)
    # all adjectives
    cnt1 = len([1 for item in pos1 if item[1].startswith('J')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('J')])
    if cnt1 == 0 and cnt2 == 0:
        t2 = 0
    else:
        t2 = (cnt1 +0.001) / (cnt2 + 0.001)
    # all adverbs
    cnt1 = len([1 for item in pos1 if item[1].startswith('R')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('R')])
    if cnt1 == 0 and cnt2 == 0:
        t3 = 0
    else:
        t3 = (cnt1 +0.001) / (cnt2+0.001)
    # all nouns
    cnt1 = len([1 for item in pos1 if item[1].startswith('N')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('N')])
    if cnt1 == 0 and cnt2 == 0:
        t4 = 0
    else:
        t4 = (cnt1 +0.001) / (cnt2 +0.001)
    # all verbs
    cnt1 = len([1 for item in pos1 if item[1].startswith('V')])
    cnt2 = len([1 for item in pos2 if item[1].startswith('V')])
    if cnt1 == 0 and cnt2 == 0:
        t5 = 0
    else:
        t5 = (cnt1+ 0.001) / (cnt2 + 0.001)

    if shorter == 2:
        t1 = 1 / (t1 + 0.001)
        t2 = 1 / (t2 + 0.001)
        t3 = 1 / (t3 + 0.001)
        t4 = 1 / (t4 + 0.001)
        t5 = 1 / (t5 + 0.001)

    return [t1, t2, t3, t4, t5]



#Resnik Similarity using Information Content from Brown Corpus

##use information content generated from the Brown corpus to compute the resnik similarity between paths in the wordnet trees for the given sentence pairs. This approach uses IC of the Least Common Subsumer (most specific ancestor node) to output a score which is used by the Support Vector Regression model.

In [ ]:
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'): return 'n'
    if tag.startswith('V'): return 'v'
    if tag.startswith('J'): return 'a'
    if tag.startswith('R'): return 'r'
    return None

def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None


def sentence_similarity_information_content(sentence1, sentence2):

    ''' compute the sentence similairty using information content from wordnet '''
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
    score, count = 0.0, 0
    ppdb_score, align_cnt = 0, 0
    # For each word in the first sentence
    for synset in synsets1:
        L = []
        for ss in synsets2:
            try:
                L.append(synset.res_similarity(ss, brown_ic))
            except:
                continue
        if L:
            best_score = max(L)
            score += best_score
            count += 1
    # Average the values
    if count >0: score /= count
    return score

#Running

##In this section a similarty score for each pair in the datasets is calculated using all above methods (except InferSent, for more info see InferSent section), Then the scores are feed to a SVR model as features. Results for the test set are stored in `results.txt`

#Preprocessing
##Used tokenization and lemmatization on the data as a pre-processing step before we turn the data into our models. Chose to do this step as lemmatization does not take away any semantic information from sentences and hence was an essential step for our application.

In [ ]:

T0 =time.time()
#----------
# training
first_sents = []
second_sents = []
true_score = []


def lower(string):
  return str.lower(string)

def normalize(corpus):
  lemmatizer = WordNetLemmatizer()
  stop_words = ["a","a's","able","about","above","according","accordingly","across","actually","after","afterwards","again","against","ain't","all","allow","allows","almost","alone","along","already","also","although","always","am","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart","appear","appreciate","appropriate","are","aren't","around","as","aside","ask","asking","associated","at","available","away","awfully","b","be","became","because","become","becomes","becoming","been","before","beforehand","behind","being","believe","below","beside","besides","best","better","between","beyond","both","brief","but","by","c","c'mon","c's","came","can","can't","cannot","cant","cause","causes","certain","certainly","changes","clearly","co","com","come","comes","concerning","consequently","consider","considering","contain","containing","contains","corresponding","could","couldn't","course","currently","d","definitely","described","despite","did","didn't","different","do","does","doesn't","doing","don't","done","down","downwards","during","e","each","edu","eg","eight","either","else","elsewhere","enough","entirely","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","f","far","few","fifth","first","five","followed","following","follows","for","former","formerly","forth","four","from","further","furthermore","g","get","gets","getting","given","gives","go","goes","going","gone","got","gotten","greetings","h","had","hadn't","happens","hardly","has","hasn't","have","haven't","having","he","he's","hello","help","hence","her","here","here's","hereafter","hereby","herein","hereupon","hers","herself","hi","him","himself","his","hither","hopefully","how","howbeit","however","i","i'd","i'll","i'm","i've","ie","if","ignored","immediate","in","inasmuch","inc","indeed","indicate","indicated","indicates","inner","insofar","instead","into","inward","is","isn't","it","it'd","it'll","it's","its","itself","j","just","k","keep","keeps","kept","know","knows","known","l","last","lately","later","latter","latterly","least","less","lest","let","let's","like","liked","likely","little","look","looking","looks","ltd","m","mainly","many","may","maybe","me","mean","meanwhile","merely","might","more","moreover","most","mostly","much","must","my","myself","n","name","namely","nd","near","nearly","necessary","need","needs","neither","never","nevertheless","new","next","nine","no","nobody","non","none","noone","nor","normally","not","nothing","novel","now","nowhere","o","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","only","onto","or","other","others","otherwise","ought","our","ours","ourselves","out","outside","over","overall","own","p","particular","particularly","per","perhaps","placed","please","plus","possible","presumably","probably","provides","q","que","quite","qv","r","rather","rd","re","really","reasonably","regarding","regardless","regards","relatively","respectively","right","s","said","same","saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible","sent","serious","seriously","seven","several","shall","she","should","shouldn't","since","six","so","some","somebody","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub","such","sup","sure","t","t's","take","taken","tell","tends","th","than","thank","thanks","thanx","that","that's","thats","the","their","theirs","them","themselves","then","thence","there","there's","thereafter","thereby","therefore","therein","theres","thereupon","these","they","they'd","they'll","they're","they've","think","third","this","thorough","thoroughly","those","though","three","through","throughout","thru","thus","to","together","too","took","toward","towards","tried","tries","truly","try","trying","twice","two","u","un","under","unfortunately","unless","unlikely","until","unto","up","upon","us","use","used","useful","uses","using","usually","uucp","v","value","various","very","via","viz","vs","w","want","wants","was","wasn't","way","we","we'd","we'll","we're","we've","welcome","well","went","were","weren't","what","what's","whatever","when","whence","whenever","where","where's","whereafter","whereas","whereby","wherein","whereupon","wherever","whether","which","while","whither","who","who's","whoever","whole","whom","whose","why","will","willing","wish","with","within","without","won't","wonder","would","would","wouldn't","x","y","yes","yet","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","z","zero"]
  normalized_corpus = []
  for s in corpus:
    normalized_s = []
    for w in s:
      if w.isdecimal() or w.isdigit():
        normalized_s.append('NUM')
      elif w in string.punctuation:
        pass
      elif w in stop_words:
        pass
      else:
        normalized_s.append(lemmatizer.lemmatize(w.lower()))
    normalized_corpus.append(normalized_s)
  return normalized_corpus


tokenizer = TweetTokenizer()


with open("new_train.txt",'r') as f:
    for line in f.readlines():
        line_split = line.split('\t')
        first_sentence = line_split[0]
        second_sentence = line_split[1]
        gs = line_split[2]
        first_sents.append(first_sentence)
        second_sents.append(second_sentence)
        true_score.append(gs)

word_tokenized = [tokenizer.tokenize(s) for s in first_sents]
first_sents_temp = normalize(word_tokenized)

first_sentence = []
for s in first_sents_temp:
  temp = ""
  for w in s:
    temp += w + " "
  first_sentence += [temp]

word_tokenized = [tokenizer.tokenize(s) for s in second_sents]
second_sents_temp = normalize(word_tokenized)

second_sents = []
for s in second_sents_temp:
  temp = ""
  for w in s:
    temp += w + " "
  second_sents += [temp]


Counts_for_tf = defaultdict(int)

for sent in first_sents:
    for w in [w.strip("?.,") for w in sent.split()]: Counts_for_tf[w] += 1
for sent in second_sents:
    for w in [w.strip("?.,") for w in sent.split()]: Counts_for_tf[w] += 1

feature_scores = []
N = len(first_sents)
T1 = time.time()
for i in range(N):
    s1 = first_sents[i]
    s2 = second_sents[i]


    scores = [ sentence_similarity_simple_baseline(s1,s2, Counts_for_tf),
                sentence_similarity_word_alignment(s1,s2)
                ,sentence_similarity_information_content(s1,s2)
                , extract_overlap_pen(s1, s2)
                ,*extract_absolute_difference(s1, s2)
                ,*extract_mmr_t(s1, s2)
              ]
    # cosine similarity
    feature_scores.append(scores)
    if 0 == (i+1) % int(N/10): print("%.2f" % ( (i +1)*1.0/ N *100), "%"+"finished (",time.time() - T1,")")

scaler = sklearn.preprocessing.StandardScaler(); scaler.fit(feature_scores); X_features = scaler.transform(feature_scores)
print("Elapsed time:",time.time() - T0,"(preprocessing)")
clf = SVR(kernel='linear')
clf.fit(X_features, true_score)

#-----------
# predicting
first_sents = []
second_sents = []
with open("new_test.txt",'r') as f_val:
    for line in f_val.readlines():
        line_split = line.split('\t')
        first_sentence = line_split[0]
        second_sentence = line_split[1]
        first_sents.append(first_sentence)
        second_sents.append(second_sentence)

word_tokenized = [tokenizer.tokenize(s) for s in first_sents]
first_sents_temp = normalize(word_tokenized)

first_sentence = []
for s in first_sents_temp:
  temp = ""
  for w in s:
    temp += w + " "
  first_sentence += [temp]

word_tokenized = [tokenizer.tokenize(s) for s in second_sents]
second_sents_temp = normalize(word_tokenized)

second_sents = []
for s in second_sents_temp:
  temp = ""
  for w in s:
    temp += w + " "
  second_sents += [temp]

for sent in first_sents:
    for w in [w.strip("?.,") for w in sent.split()]: Counts_for_tf[w] += 1
for sent in second_sents:
    for w in [w.strip("?.,") for w in sent.split()]: Counts_for_tf[w] += 1


feature_scores = []
N = len(first_sents)
T1 = time.time()
for i in range(N):
    s1 = first_sents[i]
    s2 = second_sents[i]

    scores = [ sentence_similarity_simple_baseline(s1,s2, Counts_for_tf),
                sentence_similarity_word_alignment(s1,s2)
                ,sentence_similarity_information_content(s1,s2)
                ,extract_overlap_pen(s1, s2)
                ,*extract_absolute_difference(s1, s2)
                ,*extract_mmr_t(s1, s2)
                ]

    feature_scores.append(scores)
    if 0 == (i+1) % int(N/10): print("%.2f" % ((i+1) *1.0 / N *100),"%"+"finished (",time.time() - T1,")")
X_features = scaler.transform(feature_scores)
Y_pred_np = clf.predict(X_features)
Y_pred_np = [min(5,max(0,p),p) for p in Y_pred_np]
with open("results.txt",'w+') as f_pred:
    for i in range(len(Y_pred_np)):
        f_pred.write(str(Y_pred_np[i])+'\n')
print("Elapsed time:",time.time() - T0)

9.99 %finished ( 3.9240200519561768 )
19.98 %finished ( 7.12114143371582 )
29.97 %finished ( 10.031977891921997 )
39.97 %finished ( 13.225254535675049 )
49.96 %finished ( 17.69743537902832 )
59.95 %finished ( 22.069612503051758 )
69.94 %finished ( 25.401242971420288 )
79.93 %finished ( 29.57795023918152 )
89.92 %finished ( 34.00630068778992 )
99.92 %finished ( 38.37580108642578 )
Elapsed time: 40.243180990219116 (preprocessing)
10.00 %finished ( 1.296189546585083 )
19.99 %finished ( 2.7547202110290527 )
29.99 %finished ( 3.9554872512817383 )
39.98 %finished ( 4.9682416915893555 )
49.98 %finished ( 6.041013717651367 )
59.97 %finished ( 7.044367074966431 )
69.97 %finished ( 8.072548151016235 )
79.97 %finished ( 9.045163869857788 )
89.96 %finished ( 10.005644798278809 )
99.96 %finished ( 11.002928733825684 )
Elapsed time: 58.750736951828


#Evaluation

##Using human assigned scores and predicted scores from test dataset, we then calculate pearson score

In [ ]:
import pprint
import argparse
import math
import numpy as np
from scipy.stats import pearsonr



def main():
    with open("new_test.txt",'r') as f1:
        with open("results.txt",'r') as f2:
            gold = f1.readlines()
            pred = f2.readlines()
            # http://alt.qcri.org/semeval2017/task1/
            # evaluation -- Pearson correlation
            gold = [float(g.split()[-1]) for g in gold]
            pred = [float(p.split()[-1]) for p in pred]
    # https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.pearsonr.html
    ret = pearsonr(gold,pred)[0]

    print("Performance:",ret)

main()
